# Standard normal CDF
Write a program in Python that asks a user for a number `x`, and then outputs the value of the cumulative distribution function of the standard normal distribution at `x`.
## Example

`Enter x: 0.3
The value of the standard normal CDF at 0.3 is 0.6179114221889526`

**Hint:** You'll want to use the `input(...)` and `print(...)` functions for this. Recall that `input(...)` returns a string, which must be converted to a floating point number using `float(...)`. If we do `from scipy.stats import norm`, then the cumulative distribution function of the standard normal distribution can be accessed as `norm.cdf(...)`.

In [1]:
# your answer here
from scipy.stats import norm
x = float(input('Enter x: '))
value = norm.cdf(x)
print(value)

0.6179114221889526


# Black/Scholes formula in Python
The Black/Scholes price of a European call option expiring at time $T$ with strike price $K$ is
$$
C(S,t)=SN(d_1)−Ke^{−r(T−t)}N(d_2)
$$
where $S$ is the current price of the underlying asset, $t$ the current time and $r$ the continuously compounded riskfree interest rate. $N(d)$ denotes the cumulative distribution function of the standard normal distribution, and
$$
\begin{eqnarray}
d_1 &=& \frac{\ln\frac{S}K+(r+\frac12\sigma^2)(T−t)}{\sigma\sqrt{T-t}}\\
d_2 &=& d_1−\sigma\sqrt{T-t}
\end{eqnarray}
$$
where the $\sigma$ denotes the volatility of the underlying asset.

Using the scaffold provided, write a Python program which prompts the user for the required inputs, and prints the Black/Scholes price of the option.

## Example

`Enter the underlying stock price: 100
Enter the strike price: 100
Enter the volatility: 0.3
Enter continuously compounded interest rate: 0.02
Enter the time to maturity: 2.5
The call option price is: 20.82854108910454`

In [8]:
import numpy as np
from scipy.stats import norm

stock = float(input('Enter the underlying stock price: '))
strike = float(input('Enter the strike price: '))
sigma = float(input('Enter the volatility: '))
interest = float(input('Enter continuously compounded interest rate: '))
maturity = float(input('Enter the time to maturity: '))

# To do: calculate the BlackScholes price here

d1 = (np.log(stock/strike)+(interest+0.5*sigma**2)*(maturity))/(sigma*np.sqrt(maturity))
d2 = d1 - sigma*np.sqrt(maturity)
BlackScholes = stock*norm.cdf(d1) - strike*np.exp(-interest*maturity)*norm.cdf(d2)

print('The call option price is: ' + str(BlackScholes))

The call option price is: 20.82854108910454


# Black Scholes formula as a Python function
As in the previous task, consider the Black/Scholes price of a European call option expiring at time $T$ with strike price $K$. Similarly, the price of a European put option expiring at time $T$ with strike price $K$ is
$$
P(S,t)=Ke^{−r(T−t)}N(−d_2)−SN(−d_1)
$$
Using the scaffold provided, write a Python function which calculates the Black/Scholes price of the option, where the function takes six arguments (in this order): $S$, $K$, $\sigma$, $r$, $T$ and a 1 for a call or -1 for a put.

## Example:

`Enter the underlying stock price: 100
Enter the strike price: 100
Enter the volatility: 0.3
Enter continuously compounded interest rate: 0.05
Enter the time to maturity: 2
Enter 1 for call or -1 for put option: 1
The option price is: 21.193735255280203`

## Another example (a put option):

`Enter the underlying stock price: 100
Enter the strike price: 100
Enter the volatility: 0.3
Enter continuously compounded interest rate: 0.05
Enter the time to maturity: 2
Enter 1 for call or -1 for put option: -1
The option price is: 11.677477058876157`

In [9]:
def BlackScholes(S,K,sgm,r,T,callput):
    d1 = (np.log(S/K)+(r+0.5*sgm**2)*(T))/(sgm*np.sqrt(T))
    d2 = d1 - sgm*np.sqrt(T)    
    
    if callput == 1:
        price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        return price
    elif callput == -1:
        price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
        return price
    else:
        print("Have to enter 1 or -1")
    
stock = float(input('Enter the underlying stock price: '))
strike = float(input('Enter the strike price: '))
sigma = float(input('Enter the volatility: '))
interest = float(input('Enter continuously compounded interest rate: '))
maturity = float(input('Enter the time to maturity: '))
callput = int(input('Enter 1 for call or -1 for put option: '))
print('The option price is: ')
print(BlackScholes(stock,strike,sigma,interest,maturity,callput))

The option price is: 
11.677477058876157


# Implied volatility
To calculate the price of a European option, we need to know whether it is a call or a put, the strike $K$, and its time to expiry - these are properties specified in the option contract. Furthermore, we need to know the current price of the underlying asset $(S)$ and the interest rate $r$ - these are things which can arguably be observed in the market. Lastly, we need the volatility $\sigma$ - this is not directly observable in the market. Although it can be statistically estimated, when prices for actively traded options are available in the market, the preferred method to obtain sigma is to **imply** it from those market prices. This is the <B><I>implied volatility</I></B>, i.e. the $\sigma$ which needs to be inserted into the Black/Scholes formula so that, in combination with the observed values for the other input variables, it returns the observed market price of the option.

Implied volatility cannot be obtained by algebraically solving the Black/Scholes formula for volatility - this is not analytically possible. Therefore, Black/Scholes implied volatility must be obtained numerically using a *root search.* The SciPy package provides a root search routine for Python, which for a user-defined function finds the function argument which results in a zero function value. For example, consider the function
$$
f(x)=ax+b
$$
We define this as a Python function and determine its root (for a specific choice of $a$ and $b$) by calling the SciPy function `root_scalar`:

In [10]:
from scipy import optimize
def f(x,a,b):
    return a*x+b
sol = optimize.root_scalar(f, bracket=[-1000,1000], args=(2,-5))
print(sol.root)

2.5


Note that as well as $f$, two named arguments are passed to `root_scalar`: `bracket` specifies a lower and upper bound in the search for the root, and `args` specifies the other arguments ($a$ and $b$) of $f$. Feel free to run and play with the above code (for example by modifying `args`, or even the function $f$).

## Task
Using the scaffold provided, write a Python function which calculates the Black/Scholes implied volatility of an option, where the function takes six arguments (in this order): $S$, $K$, $r$, $T$, a 1 for a call or -1 for a put, and the option price. Note that you will need to define additional functions in order to conduct the root search.

### Example output

`Enter the underlying stock price: 100 
Enter the strike price: 100 
Enter continuously compounded interest rate: 0.05 
Enter the time to maturity: 2 
Enter 1 for call or -1 for put option: 1 
Enter the option price: 22.7251160722058 
The implied volatility is: 0.3300000000000001`

In [13]:
from scipy import optimize
def BlackScholesImpVol(S,K,r,T,callput,price):
    
    def d(x,S,K,r,T):
        d1 = (np.log(S/K)+(r+0.5*x**2)*(T))/(x*np.sqrt(T))
        d2 = d1 - x*np.sqrt(T)
        return d1, d2
    
    def f(x,S,K,r,T,p,callput):
        d1, d2 = d(x,S,K,r,T)
        
        if callput == 1:
            return S*norm.cdf(d1)-K*np.exp(-r*(T))*norm.cdf(d2)-p
        else:
            return K*np.exp(-r*T)*norm.cdf(-d2)-S*norm.cdf(-d1)-p
    
    sol = optimize.root_scalar(f, bracket=[-10000.000000, 10000.000000],
                           args=(S, K, r, T, price, callput))
    return sol

stock = float(input('Enter the underlying stock price: '))
strike = float(input('Enter the strike price: '))
interest = float(input('Enter continuously compounded interest rate: '))
maturity = float(input('Enter the time to maturity: '))
callput = int(input('Enter 1 for call or -1 for put option: '))
price = float(input('Enter the option price: '))
print('The implied volatility is: ')
print(BlackScholesImpVol(stock,strike,interest,maturity,callput,price))

The implied volatility is: 
      converged: True
           flag: 'converged'
 function_calls: 21
     iterations: 20
           root: 0.33000000000000007


# Binomial option pricing model - European call option
In the binomial option pricing model introduced in the first lecture, the price of a European call option is given by
$$
C=\frac1{R^n}\sum_{i=a}^n\left(\begin{array}{c}n\\ i\end{array}\right)p^i(1−p)^{n−i}(u^id^{n−i}S−K)
$$
where $S$ is the initial stock price, $K$ is the strike price, $u$ is the "up" factor, $d$ is the "down" factor, $R$ is the accummulation factor, $p$ is the risk-neutral probability of an "up" move, $n$ is the number of time steps until expiry of the option, and $a$ is the minimum number of "up" moves such that
$$
u^ad^{n−a}S>K
$$
Note that the binomial probability,
$$
\left(\begin{array}{c}n\\ i\end{array}\right)p^i(1−p)^{n−i}
$$
can be calculated in Python using the function `binom.pmf(i,n,p)` from the SciPy Stats package, as follows:

In [14]:
from scipy.stats import binom
n = 10
i = 3
p = 0.5
print(binom.pmf(i,n,p))

0.11718750000000014


Using the scaffold provided, write a Python function `BinomialCall()` which calculates the price of a European call option in the binomial model, where the function takes six arguments (in this order): $S$, $K$, $u$, $d$, $R$, $n$.

## Example:
This example corresponds to the one given on Slides 35 and 36 of Lecture 1b:

`Enter the underlying stock price: 100 
Enter the strike price: 90 
Enter the up factor: 1.2 
Enter the down factor: 0.8 
Enter the accumulation factor: 1.1 
Enter the number of time steps: 2 
The option price is: 26.96280991735539`

In [15]:
from scipy.stats import binom
def BinomialCall(S, K, u, d, Acc, N):                                                        
    
    R = Acc    
    p = (R - d)/ (u - d)  
    disc = 1/R**N
    
    f_value = 0
    
    for i in range(1, N+1):
        prob = binom.pmf(i,N,p)
        f_value += prob * ((u**i) * (d**(N-i)) * S - K)
        d_value = disc*f_value
    return d_value
    
stock = float(input('Enter the underlying stock price: '))
strike = float(input('Enter the strike price: '))
u = float(input('Enter the up factor: '))
d = float(input('Enter the down factor: '))
R = float(input('Enter the accumulation factor: '))
N = int(input('Enter the number of time steps: '))
print('The option price is: ')
print(BinomialCall(stock,strike,u,d,R,N))

The option price is: 
26.962809917355393


# Binomial option pricing model - American put option
Using the scaffold provided, write a Python function `BinomialAmericanPut()` which calculates the price of an American put option in the binomial model, where the function takes six arguments (in this order): $S$, $K$, $u$, $d$, $R$, $n$. These parameters have the same meaning as in the previous task. Note that unlike in the previous task, there is no direct pricing formula for the American put - the price must be calculated by iterating step by step backwards through the binomial lattice, as illustrated in the spreadsheet `Binomial.xlsx` created in the lecture (and available on the Canvas site).

## Example:
This example corresponds to the one given on Slides 42 of Lecture 1:

`Enter the underlying stock price: 120
Enter the strike price: 95
Enter the up factor: 1.2
Enter the down factor: 0.8
Enter the accumulation factor: 1.1
Enter the number of time steps: 3
The option price is: 1.1401202103681396`

In [16]:
import numpy as np
def BinomialAmericanPut(S, K, u, d, Acc, N):
    
    R = Acc    
    p = (R - d)/ (u - d)  
    q = 1.0 - p   
    disc = 1/(Acc**N)
    
    S_T = np.array( [(S * u**j * d**(N - j)) for j in range(N + 1)] ) 
    
    V = np.zeros(N+1)
    for j in range(0, N+1):
        V[j] = max(0, K - S_T[j]) 
    
    for i in np.arange(N-1,-1,-1):
        for j in range(0,i+1):
            V[j] = 1/(Acc) * ( p*V[j+1] + q*V[j] )
            S_T = S * u**j * d**(i-j)
            V[j] = max(V[j], K - S_T)
            
    return V[0]

stock = float(input('Enter the underlying stock price: '))
strike = float(input('Enter the strike price: '))
u = float(input('Enter the up factor: '))
d = float(input('Enter the down factor: '))
R = float(input('Enter the accumulation factor: '))
N = int(input('Enter the number of time steps: '))
print('The option price is: ')
print(BinomialAmericanPut(stock,strike,u,d,R,N))

The option price is: 
1.1401202103681387
